## Polynom

Polynomem $p(x)$ stupně $n$ nazýváme výraz tvaru

$$
p(x) = \sum_{i=0}^n a_i x^i = a_0 + a_1 \cdot x + \ldots + a_n \cdot x^n
$$
a čísla $a_i \in \mathcal{R},\ i=1,\dots n$ se nazývají koeficienty.

Napište funkci, která vyhodnotí polynom zadaný n-ticí koeficientů v bodě x, tj.
```python
# polynom(x, 3, 2, 1) vyhodnoti 3 + 2x + x^2
polynom(1, 3, 2, 1)
>> 6
```

Je vhodné pro zápis polynomu použít tzv. Hornerovo schéma (ušetří mnoho operací a snáze se implementuje). Pro polynom stupně 3 vypadá takto:
$$
a_0 + a_1\cdot x + a_2 \cdot x^2 + a_3 \cdot x^3 = a_0 + x \cdot ( a_1 + x \cdot ( a_2 + a_3 \cdot x) )
$$

In [ ]:
def polynom(x, *coefs):
    val = coefs[-1]
    for c in coefs[-2::-1]:
        val = val * x + c
    return val

polynom(1,3,2,1)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

sin = (0, 1, 0, -1/6., 0, 1/120., 0, -1./5020)
cos = (1, 0, -1./2, 0, 1/24, 0, -1./720, 0., 1./40320)
exp = [abs(cos[i]) if i%2==0 else abs(sin[i]) for i in range(len(cos))]

x = np.linspace(-3, 3, 201)
y = polynom(x, *sin)
plt.plot(x,y)
plt.plot(x,np.sin(x))
plt.plot(x, polynom(x, *cos))
plt.plot(x, polynom(x, *exp))
plt.plot(x, np.exp(x))
plt.show()

## zapomenuté věci

- break a continue
- script file

## Vlastní modul

In [ ]:
import module

print(module.__name__)
dir(module)

In [ ]:
module.function()

In [ ]:
from module import function as mfunc

mfunc()

In [ ]:
from package.module_a import *
import package.module_b as mb

function_a()
mb.function_b()

## `__name__`

In [ ]:
import package.module_b as mb

mb.__name__

In [ ]:
from potentials import *

## dekoratory

- umoznuji rozsirovat opakovany zpusobem funkce
- dekorator ma jako vstup funkci a vraci funkci. Obali jeji prubeh do dalsiho kodu
- muzeme vyrabet ruzne dekoratory v zavislosti na dalsich parametrech. Je to docela uzitecne, celkem neprehledne a hlavne se tomu rika "metaprogramovani"

In [ ]:
def dec(func):
    def wrapper():
        print("calling function", func.__name__)
        func()
    return wrapper

@dec
def f():
    print("Hallo")
    
# stejne jako:
# f = func(f)

f()

In [ ]:
@dec
def add(x, y):
    return x + y

add(1, 2)

In [ ]:
def dec(func):
    def wrapper(*args, **kwargs):
        print("calling function", func.__name__)
        return func(*args, **kwargs)
    return wrapper

def add(x, y):
    return x + y

decorated = dec(add)

decorated(1,2)

@dec
def subtract(x, y):
    return x - y

subtract(2, 1)

In [ ]:
## dekoratory s parametry

# dekorator - funkce, ktera vraci funkci.
# ma-li byt dekorator zavisly na nejakem parametru, potrebujeme, funkci, ktera ho vyrobi.
# tedy funkci, kterat vraci funkci, ktera vraci funkci

def underline(width=50):
    def dec(func):
        def wrapper():
            func()
            print("-" * width)
        return wrapper
    return dec

@underline(20)
def print_header():
    print("this is the header")
    
print_header()

In [ ]:
LOG_INFO    = 0
LOG_WARNING = 1
LOG_DEBUG   = 2

LOG_STR_LST = ["INFO", "WARNING", "DEBUG"]

log_level = LOG_DEBUG

def log(level = LOG_INFO):
    def dec(func):
        def wrapper(*args, **kwargs):
            if level <= log_level:
                print("{}: running function: {}".format(LOG_STR_LST[level], func.__name__))
                if log_level >= LOG_DEBUG:
                    print("\targs:", args)
                    print("\tkwargs:", kwargs)
            return func(*args, **kwargs)
        return wrapper
    return dec

@log(LOG_INFO)
def add(x, y):
    return x + y

@log(LOG_WARNING)
def do_warning_level_stuff():
    pass

@log(LOG_DEBUG)
def do_debug_level_stuff(**kwargs):
    pass

do_debug_level_stuff(neco = True)
add(1, 2)
do_warning_level_stuff()

## Nestihlo se na hodině

Existuje řada algoritmů, které dokáží seřadit nějakou posloupnost. Některé jsou lepší, některé horší, ale to znáte z jiných předmětů. Podívejme se spíš na to, jak ty algoritmy implementovat, aby byly snadno použitelné. Ukážeme si to na selection sort, bubble sort a random sort (můj oblíbenec).

Napřed úplně přímočará implementace

In [ ]:
import random as r

def selection_sort(lst):
    n = len(lst)
    for i in range(n):
        jmin = i
        for j in range(i, n):
            if lst[j] < lst[jmin]:
                jmin = j
        lst[i], lst[jmin] = lst[jmin], lst[i]
    return lst

def bubble_sort(lst):
    n = len(lst)
    for i in range(n):
        for j in range(n-i-1):
            if lst[j] > lst[j+1]:
                lst[j], lst[j+1] = lst[j+1], lst[j]
    return lst

def random_sort(lst):
    lst_sorted = False
    n = len(lst)
    while not lst_sorted: 
        lst_sorted = True
        for i in range(n-1):
            if lst[i+1] < lst[i]:
                lst_sorted = False
                break
                
        if not lst_sorted:
            r.shuffle(lst)
            
    return lst

In [ ]:
lst = [5, 2, 4, 1, 3]
print(selection_sort(lst), bubble_sort(lst), random_sort(lst))

Všechno funguje, ale má to několik slabin:
    
1. některé pasáže jsou ošklivé
2. neumíme snadno otočit pořadí
3. umíme seřadit jenom prvky, které umíme

Bod 1. vyřešíme zavedením pomocných funkcí jako

`swap` - zamění prvky v kolekci
`is_sorted` - zkontroluje, zda je kolekce seřazená
`argmin` - najde index nejmenšího prvku v kolekci

Body 2. a 3. můžeme vyřešit tím, že každému algoritmu předáme funkci `compare`, kterou nahradíme veškeré porovnání. Dohodněme se, že funkci `compare(x, y)` vrátí `True`, pokud `x>y`.

In [ ]:
import random as r

def swap(lst, i, j):
    lst[i], lst[j] = lst[j], lst[i]
    
def is_sorted(lst, compare):
    for x, y in zip(lst, lst[1:]):
        if compare(x, y):
            return False
    return True

def argmin(lst, compare):
    arg = 0
    val = lst[0]
    for i, x in enumerate(lst):
        if compare(val, x):
            val = x
            arg = i
    return arg

def selection_sort(lst, compare):
    n = len(lst)
    for i, x in enumerate(lst):
        j = argmin(lst[i:], compare) + i
        swap(lst, i, j)
    return lst

def bubble_sort(lst, compare):
    n = len(lst)
    for i in range(n):
        for j in range(n-i-1):
            if compare(lst[j], lst[j+1]):
                swap(lst, j, j+1)
    return lst

def random_sort(lst, compare):
    while not is_sorted(lst, compare):
        r.shuffle(lst)
    return lst

In [ ]:
def compare(x, y):
    return x > y

lst = [5, 2, 4, 1, 3]
print(selection_sort(lst, compare), bubble_sort(lst, compare), random_sort(lst, compare))

Použití funkce je teď o něco složitější, ale funkce je mnohem variabilnější. Můžeme si vybírat, co se jak bude řadit. Ruzné algoritmy navíc můžeme zabalit do obecnější funkce sort, např takto:

In [ ]:
def sort(lst, compare = lambda x,y: x>y, method = "selection"):
    if method == "selection":
        return selection_sort(lst, compare)
    elif method == "bubble":
        return bubble_sort(lst, compare)
    elif method == "random":
        return random_sort(lst, compare)
    else:
        print("unknown method")
        return

In [ ]:
# obycejne serazeni cisel pomoci bubble sort

sort([3,1,2], method="bubble")

In [ ]:
# obracene serazeni cisel pomoci random sort

sort([3,1,2], compare = lambda x, y: x<y, method="random")

In [ ]:
# razeni dle delky vs abecedni razeni

sort(["aaa", "c", "bb"], lambda x,y: len(x)>len(y)), sort(["aaa", "c", "bb"], lambda x,y: x>y)

In [ ]:
# serazeni s vyuzitim dictionary

prices = {
    "klobasa" : 40,
    "jogurt" : 20,
    "rohlik" : 3
}


nakup = ["klobasa", "rohlik", "jogurt"]
sort(nakup, lambda a,b: prices[a] > prices[b])
print(nakup)